In [1]:
immutable Transformation
    self :: Function
    deriv :: Function
end

import Core.eval

function eval(transformation :: Transformation)
    transformation.self
end

function deriv(transformation :: Transformation)
    transformation.deriv
end

function sigmoid(x)
        1. / (1. + exp(-x)) - 0.5
end

function dsigmoid(x)
    exp(-x) / (1. + exp(-x)) ^ 2
end

const Sigmoid = Transformation(sigmoid, dsigmoid)

const Identity = Transformation(identity, one) 

Transformation(identity,one)

In [2]:
function calc_norm(array :: Matrix{Float64})
  sqrt(sumabs2(array) / size(array, 1))
end

function calc_norm(vector :: Array{Float64, 1})
  sqrt(sumabs2(vector))
end

function restriction(vector)
  vector / (calc_norm(vector) + 0.001)
end

function init(a...)
    restriction(randn(a...))
end

const DefaultRate = 0.01

immutable Class
    class_name :: String
    type_len :: Int64
end

type Object
    class :: Class
    value :: Array{Float64, 1}
end

Object(class :: Class) = Object(class, init(class.type_len))

type DFunction
    in_classes :: Array{Class, 1}
    out_class :: Class
    f_matrix :: Matrix{Float64}
    f_transformation :: Transformation
    learning_rate :: Float64
end

function LFunction(in_classes :: Array{Class, 1}, out_class :: Class, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(in_classes, out_class, init(out_len, in_len), Identity, learning_rate)
end

function SFunction(in_classes :: Array{Class, 1}, out_class :: Class, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(in_classes, out_class, init(out_len, in_len), Sigmoid, learning_rate)
end

function DFunction(in_classes :: Array{Class, 1}, out_class :: Class, f_transformation = Sigmoid, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(in_classes, out_class, init(out_len, in_len), f_transformation, learning_rate)
end

function CoFunction(in_class :: Class, f_transformation = Sigmoid)
    DFunction([in_class], in_class, diagm(ones(in_class.type_len)), f_transformation, 0.0)
end

function apply(func :: DFunction, objs :: Array{Object, 1})
    inputs = vcat([obj.value for obj in objs]...)
    outputs = eval(func.f_transformation).(func.f_matrix * inputs)
    Object(func.out_class, outputs)
end

function apply!(func :: DFunction, in_objs :: Array{Object, 1}, out_obj :: Object)
    inputs = vcat([obj.value for obj in in_objs]...)
    out_obj.value .= eval(func.f_transformation).(func.f_matrix * inputs)
end

apply! (generic function with 1 method)

In [3]:
Sensor = Class("Sensor", 100)
sensor = Object(Sensor)
Action = Class("Action", 120)
action = Object(Action)
act = DFunction([Sensor, Sensor, Action], Action)
apply(act, [sensor, sensor, action])

Object(Class("Action",120),[-0.00989273,0.000711335,-0.00982096,0.00317703,-0.00214918,0.00346893,-0.00395179,0.00698046,0.0223216,0.0441083  …  -0.0324586,0.0257685,-0.0154239,-0.013352,-0.0244933,0.0394573,-0.000261382,0.0380902,0.00641128,-0.0030648])

In [4]:
type Tree
    op
    value
    subtrees :: Array{Tree, 1}
end

function _bottom_up(func :: Function, tree :: Tree, dict :: Dict)
    function f()
        func(tree, [_bottom_up(func, t, dict) for t in tree.subtrees])
    end
    get!(f, dict, tree)
end

## bottom_up(func :: Function, tree :: Tree) = _bottom_up(func, tree, Dict())

function bottom_up(func :: Function, tree :: Tree)
    func(tree, [bottom_up(func, t) for t in tree.subtrees])
end

bottom_up (generic function with 1 method)

In [5]:
function lens_split(x :: Array, lens :: Array{Int64, 1})
    ind = cumsum(lens)
    n = length(ind)
    ind = [0; ind]
    [view(x, (ind[i]+1):ind[i+1]) for i in 1:n]
end

function init_node!(op :: Object, value :: Dict)
    value[:value] = Array{Float64, 1}(op.class.type_len)
    value[:d] = Array{Float64, 1}(op.class.type_len)
end

function init_node!(op :: DFunction, value :: Dict)
    value[:pre_matrix] = Array{Float64, 1}(size(op.f_matrix, 2))
    value[:pre_transform] = Array{Float64, 1}(op.out_class.type_len)
    value[:value] = Array{Float64, 1}(op.out_class.type_len)
    value[:inputs] = lens_split(value[:pre_matrix], [c.type_len for c in op.in_classes])
    value[:d] = Array{Float64, 1}(op.out_class.type_len)
    value[:pre_transform_d] = Array{Float64, 1}(op.out_class.type_len)
    value[:pre_matrix_d] = Array{Float64, 1}(size(op.f_matrix, 2))
    value[:input_ds] = lens_split(value[:pre_matrix_d], [c.type_len for c in op.in_classes])
end

function _init_tree!(tree :: Tree, _)
    init_node!(tree.op, tree.value)
    tree
end
    
function init_tree!(tree :: Tree)
    bottom_up(_init_tree!, tree)
end

function eval_node!(op :: Object, value :: Dict, value_list)
    value[:value] .= op.value
end

function eval_node!(op :: DFunction, value :: Dict, value_list)
    i = 1
    for v in value_list
        value[:inputs][i] .= v[:value]
        i += 1
    end
    ## value[:pre_matrix] .= vcat([v[:post_feature] for v in value_list]...)
    A_mul_B!(value[:pre_transform], op.f_matrix, value[:pre_matrix])
    value[:value] .= eval(op.f_transformation).(value[:pre_transform])
end

function _eval_tree!(tree :: Tree, _)
    eval_node!(tree.op, tree.value, (t.value for t in tree.subtrees))
    tree
end
    
## function eval_tree!(tree :: Tree)
##     bottom_up(_eval_tree!, tree)
## end
    
function eval_tree!(tree :: Tree)
    foreach(eval_tree!, tree.subtrees)
    eval_node!(tree.op, tree.value, (t.value for t in tree.subtrees))
    tree
end

eval_tree! (generic function with 1 method)

In [6]:
Sensor = Class("Sensor", 100)
sensor = Object(Sensor)
Action = Class("Action", 120)
action = Object(Action)
act = DFunction([Sensor, Action], Action)
tree = Tree(act, Dict(), [Tree(sensor, Dict(), []), Tree(action, Dict(), [])])
init_tree!(tree)
eval_tree!(tree).value[:value] - apply(act, [sensor, action]).value

120-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
function bp_transformation!(transformation, inputs, d, new_d)
    new_d .= deriv(transformation).(inputs)
    for i in 1:length(d)
        new_d[i] *= d[i]
    end
end

function bp_matrix!(inputs, matrix, d, step, new_d)
    ## step = 0.01
    ## dmatrix = reshape(d, (length(d), 1)) * reshape(inputs, (1, length(inputs)))
    ## matrix[:, :] += step * reshape(d, (length(d), 1)) * reshape(inputs, (1, length(inputs)))
    for j in 1:size(matrix, 2)
        new_d[j] = 0.0
        for i in 1:size(matrix, 1)
            matrix[i, j] += step * d[i] * inputs[j]
            new_d[j] += matrix[i, j] * d[i]
        end
    end
end

function bp_function!(op :: DFunction, value :: Dict)
    bp_transformation!(op.f_transformation, value[:pre_transform], value[:d], value[:pre_transform_d])
    bp_matrix!(value[:pre_matrix], op.f_matrix, value[:pre_transform_d], op.learning_rate, value[:pre_matrix_d])
end

function bp_tree!(tree :: Tree)
    if typeof(tree.op) == DFunction
        bp_function!(tree.op, tree.value)
        for i in 1:length(tree.subtrees)
            tree.subtrees[i].value[:d] .= tree.value[:input_ds][i]
            bp_tree!(tree.subtrees[i])
        end
        ## ds = lens_split(d, [c.feature_len for c in tree.op.in_classes])
        ## for i in 1:length(tree.subtrees)
        ##     bp_tree!(tree.subtrees[i], tree.value[:ds][i])
        ## end
        ## dds = map(bp_class!, tree.op.in_classes, [t.value for t in tree.subtrees], tree.value[:ds])
        ## foreach(bp_tree!, tree.subtrees, dds)
    end
end

bp_tree! (generic function with 1 method)

In [8]:
Sensor = Class("Sensor", 100)
sensor = Object(Sensor)
Action = Class("Action", 120)
action = Object(Action)
act = DFunction([Sensor, Action], Action)
tree = Tree(act, Dict(), [Tree(sensor, Dict(), []), Tree(action, Dict(), [])])
init_tree!(tree)
d = ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]
for i in 1:5000
    tree.value[:d] .= ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]
    bp_tree!(tree)
end
maximum(abs(ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]))

0.5199747764908644

In [9]:
function toTree(op)
    Tree(op, Dict(), [])
end

function toTree(skeleton :: Array)
    op = skeleton[1]
    subs = skeleton[2:end]
    Tree(op, Dict(), [toTree(s) for s in subs])
end

function add!(dict :: Dict, dict1 :: Dict)
    for key in keys(dict1)
        dict[key] = vcat(get!(dict, key, []), dict1[key])
    end
    dict
end

function _index(tree :: Tree, inds :: Array)
    ind = Dict{Any, Any}(tree.op => [tree])
    for ind1 in inds
        add!(ind, ind1)
    end
    ind
end

index(tree :: Tree) = bottom_up(_index, tree)

type Axiom
    tree1 :: Tree
    tree2 :: Tree
    index :: Dict
end

Axiom(tree1 :: Tree, tree2 :: Tree) = Axiom(tree1, tree2, add!(index(tree1), index(tree2)))

Axiom(skeleton1, skeleton2) = Axiom(toTree(skeleton1), toTree(skeleton2))

function push!(index :: Dict, ops :: Array)
    n = length(ops)
    for i in 1:n
        for t in index[i]
            t.op = ops[i]
        end
    end
end

function push!(index :: Dict, ops :: Dict)
    for key in keys(ops)
        if haskey(index, key)
            ts = index[key]
            for t in ts
                t.op = ops[key]
            end
        end
    end
end

function push!(axiom :: Axiom, ops)
    push!(axiom.index, variables)
end

function train!(axiom :: Axiom, variables, n = 1)
    push!(axiom.index, variables)
    init_tree!(axiom.tree1)
    init_tree!(axiom.tree2)
    d1 :: Array{Float64, 1} = axiom.tree1.value[:d]
    d2 :: Array{Float64, 1} = axiom.tree2.value[:d]
    v1 :: Array{Float64, 1} = axiom.tree1.value[:value]
    v2 :: Array{Float64, 1} = axiom.tree2.value[:value]
    for i in 1:n
        eval_tree!(axiom.tree2)
        eval_tree!(axiom.tree1)
        for j in 1:length(d1)
                d1[j] = v2[j] - v1[j]
                d2[j] = -d1[j]
        end
        bp_tree!(axiom.tree1)
        bp_tree!(axiom.tree2)
    end
end

train! (generic function with 2 methods)

In [10]:
Sensor = Class("Sensor", 100)
sensor = Object(Sensor)
Action = Class("Action", 120)
action = Object(Action)
act = DFunction([Sensor], Action)
invact = DFunction([Action], Sensor)
axiom = Axiom([1, [2, 3]], 3)

Axiom(Tree(1,Dict{Any,Any}(),Tree[Tree(2,Dict{Any,Any}(),Tree[Tree(3,Dict{Any,Any}(),Tree[])])]),Tree(3,Dict{Any,Any}(),Tree[]),Dict{Any,Any}(Pair{Any,Any}(2,Any[Tree(2,Dict{Any,Any}(),Tree[Tree(3,Dict{Any,Any}(),Tree[])])]),Pair{Any,Any}(3,Any[Tree(3,Dict{Any,Any}(),Tree[]),Tree(3,Dict{Any,Any}(),Tree[])]),Pair{Any,Any}(1,Tree[Tree(1,Dict{Any,Any}(),Tree[Tree(2,Dict{Any,Any}(),Tree[Tree(3,Dict{Any,Any}(),Tree[])])])])))

In [11]:
@time train!(axiom, [invact, act, sensor], 10000)
@time foreach(i -> eval_tree!(axiom.tree1), 1:10000)
maximum(abs(eval_tree!(axiom.tree2).value[:value] - eval_tree!(axiom.tree1).value[:value]))
Profile.clear()
@profile train!(axiom, [invact, act, sensor], 1000)
## Profile.print()
## Profile.print(format = :flat)

  1.979595 seconds (212.49 k allocations: 4.385 MB, 0.63% gc time)
  0.289891 seconds (91.39 k allocations: 1.437 MB)


In [12]:
## Lambda Calculus

Variable = Class("Variable", 100)
LambdaTerm = Class("LambdaTerm", 200)
var2term = DFunction([Variable], LambdaTerm)
app = DFunction([LambdaTerm, LambdaTerm], LambdaTerm)
lambda = DFunction([Variable, LambdaTerm], LambdaTerm)

## substitution
## t1 [x = t2]
subs = DFunction([LambdaTerm, Variable, LambdaTerm], LambdaTerm)
## x[x := N]       ≡ N
axiom_s1 = Axiom([subs, [var2term, :v], :v, :t], :t)
## y[x := N]       ≡ y, if x ≠ y
axiom_s2 = Axiom([subs, [var2term, :v1], :v2, :t], [var2term, :v1])
## (M1 M2)[x := N] ≡ (M1[x := N]) (M2[x := N])
axiom_s3 = Axiom([subs, [app, :t1, :t2], :v, :t], [app, [subs, :t1, :v, :t], [subs, :t2, :v, :t]])
## (λx.M)[x := N]  ≡ λx.M
axiom_s4 = Axiom([subs, [lambda, :v, :t1], :v, :t2], [lambda, :v, :t1])
## (λy.M)[x := N]  ≡ λy.(M[x := N]), if x ≠ y, provided y ∉ FV(N)
axiom_s5 = Axiom([subs, [lambda, :v1, :t1], :v2, :t2], [lambda, :v1, [subs, :t1, :v2, :t2]])

## beta-reduction of  ((λV.E) E′)  is E[V := E′].
axiom_beta = Axiom([app, [lambda, :v, :t], :t1], [subs, :t, :v, :t1])
## eta-reduction
axiom_eta = Axiom([lambda, :v, [app, :t, [var2term, :v]]], :t)

Axiom(Tree(DFunction(Class[Class("Variable",100),Class("LambdaTerm",200)],Class("LambdaTerm",200),[-0.0135713 -0.0111923 … -0.0475288 0.102797; 0.0159264 0.0285803 … 0.0673325 -0.0282245; … ; 0.0428357 -0.0668975 … 0.0351212 -0.0688542; 0.0429872 -0.0919481 … 0.00540028 -0.00327438],Transformation(sigmoid,dsigmoid),0.01),Dict{Any,Any}(),Tree[Tree(:v,Dict{Any,Any}(),Tree[]),Tree(DFunction(Class[Class("LambdaTerm",200),Class("LambdaTerm",200)],Class("LambdaTerm",200),[0.078969 0.00264338 … 0.0609423 -0.0125844; -0.0510452 0.00293141 … -0.0263179 -0.0278305; … ; 0.00854411 -0.0566449 … 0.0751089 -0.0558558; -0.031944 0.00303537 … -0.0122896 0.0392358],Transformation(sigmoid,dsigmoid),0.01),Dict{Any,Any}(),Tree[Tree(:t,Dict{Any,Any}(),Tree[]),Tree(DFunction(Class[Class("Variable",100)],Class("LambdaTerm",200),[-0.258278 0.17852 … 0.0488142 0.196751; 0.00900318 -0.0114557 … -0.0845612 -0.0305385; … ; -0.0230711 0.064504 … -0.00179347 -0.215298; 0.0606277 -0.0363926 … -0.0761296 0.0305013],T

In [13]:
dict = Dict()
dict[:v] = Object(Variable)
dict[:v1] = Object(Variable)
dict[:v2] = Object(Variable)
dict[:t] = Object(LambdaTerm)
dict[:t1] = Object(LambdaTerm)
dict[:t2] = Object(LambdaTerm)

Object(Class("LambdaTerm",200),[-0.0285769,0.042028,-0.0510516,0.117618,-0.146887,-0.0117117,0.0435173,-0.0833209,0.0544871,0.0132202  …  0.122447,-0.0896062,0.116571,-0.0201273,0.1169,0.0500459,0.0637783,-0.015164,-0.00209812,-0.0304327])

In [14]:
@time train!(axiom_s5, dict, 1000)

  2.210167 seconds (54.40 k allocations: 1.306 MB)


In [19]:
type Pair
    class :: Class
    
    pair :: DFunction
    first :: DFunction
    second :: DFunction
    first! :: DFunction
    second! :: DFunction
    
    axiom_first :: Axiom
    axiom_second :: Axiom
    axiom_first! :: Axiom
    axiom_second! :: Axiom
end

function Pair(class1 :: Class, class2 :: Class, type_len :: Int64)
    post_name = string("_", class1.class_name, "_", class2.class_name)
    pair_name = string("Pair", post_name)
    pclass = Class(pair_name, type_len)
    
    pair = DFunction([class1, class2], pclass)
    first = DFunction([pclass], class1)
    second = DFunction([pclass], class2)
    first! = DFunction([pclass, class1], pclass)
    second! = DFunction([pclass, class2], pclass)
    
    axiom_first = Axiom([first, [pair, :x, :y]], :x)
    axiom_second = Axiom([second, [pair, :x, :y]], :y)
    axiom_first! = Axiom([first!, :xy, :x], [pair, :x, [second, :xy]])
    axiom_second! = Axiom([second!, :xy, :y], [pair, [first, :xy], :y])
    
    Pair(pclass, pair, first, second, first!, second!, axiom_first, axiom_second, axiom_first!, axiom_second!)
end

@time p = Pair(Sensor, Action, 70)

  0.039926 seconds (20.07 k allocations: 1.706 MB)


Pair(Class("Pair_Sensor_Action",70),DFunction(Class[Class("Sensor",100),Class("Action",120)],Class("Pair_Sensor_Action",70),[0.0337646 -0.0690434 … 0.0271811 -0.046406; -0.00755438 0.152953 … 0.0732673 -0.0312492; … ; 0.115982 -0.0182171 … 0.0364762 0.141189; -0.0659663 -0.147159 … 0.0738494 -0.0318756],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("Pair_Sensor_Action",70)],Class("Sensor",100),[0.289884 -0.0185713 … -0.00250749 -0.0863891; 0.140652 0.103434 … -0.285008 0.0266675; … ; 0.054545 0.09965 … 0.0781973 -0.0397117; 0.0616697 0.107472 … 0.0337716 -0.0266583],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("Pair_Sensor_Action",70)],Class("Action",120),[-0.225809 0.100341 … 0.0660144 0.0326232; -0.111896 -0.185515 … -0.0428772 -0.107698; … ; -0.111966 0.019644 … 0.058352 0.0821587; -0.0949172 -0.168639 … -0.0498104 -0.0105251],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("Pair_Sensor_Action",70),Class("Sensor",100)],Class("Pair_Sensor_A

In [17]:
type List
    class :: Class
    
    empty :: Object

    cons :: DFunction
    first :: DFunction
    rest :: DFunction
    first! :: DFunction
    rest! :: DFunction
    
    axiom_first :: Axiom
    axiom_rest :: Axiom
    axiom_first! :: Axiom
    axiom_rest! :: Axiom
end

function List(class :: Class, type_len :: Int64)
    list_name = string("List", "_", class.class_name)
    lclass = Class(list_name, type_len)
    
    empty = Object(lclass)
    empty.value .= 0.
    
    cons = DFunction([class, lclass], lclass)
    first = DFunction([lclass], class)
    rest = DFunction([lclass], lclass)
    first! = DFunction([lclass, class], lclass)
    rest! = DFunction([lclass, lclass], lclass)
    
    axiom_first = Axiom([first, [cons, :x, :xs]], :x)
    axiom_rest = Axiom([rest, [cons, :x, :xs]], :xs)
    axiom_first! = Axiom([first!, :xxs, :x], [cons, :x, [rest, :xxs]])
    axiom_rest! = Axiom([rest!, :xxs, :xs], [cons, [first, :xxs], :xs])
    
    List(lclass, empty, cons, first, rest, first!, rest!, axiom_first, axiom_rest, axiom_first!, axiom_rest!)
end

@time l = List(Sensor, 70)

  0.044888 seconds (22.00 k allocations: 1.621 MB)


List(Class("List_Sensor",70),Object(Class("List_Sensor",70),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),DFunction(Class[Class("Sensor",100),Class("List_Sensor",70)],Class("List_Sensor",70),[-0.0478205 0.210448 … -0.0367765 -0.0568566; 0.0687008 0.0680562 … -0.00742547 0.0326543; … ; 0.0354132 -0.0833955 … 0.00737178 0.0304305; -0.0665069 0.0519551 … 0.0926316 0.0435637],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("List_Sensor",70)],Class("Sensor",100),[-0.0300245 0.0874222 … -0.0889879 0.0131396; -0.0296792 -0.0114043 … -0.113019 -0.1596; … ; -0.164922 -0.00853376 … 0.0733155 0.0825224; 0.0749854 0.0309507 … 0.164426 -0.0965762],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("List_Sensor",70)],Class("List_Sensor",70),[-0.0870762 0.0205628 … 0.0369546 -0.117241; -0.0436353 -0.00401807 … 0.00348527 0.0322049; … ; 0.0955265 -0.0878023 … 0.147869 0.161563; -0.0681513 0.0393811 … -0.0267199 0.0108862],Transformation(s

In [22]:
## Binding, Frame and Environment

Binding = Pair(Variable, LambdaTerm, 100)
Frame = List(Binding.class, 100)
Environment = List(Frame.class, 200)


List(Class("List_List_Pair_Variable_LambdaTerm",200),Object(Class("List_List_Pair_Variable_LambdaTerm",200),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),DFunction(Class[Class("List_Pair_Variable_LambdaTerm",100),Class("List_List_Pair_Variable_LambdaTerm",200)],Class("List_List_Pair_Variable_LambdaTerm",200),[0.0219447 0.00255691 … 0.109151 0.0401103; 0.0106217 0.00593834 … 0.0782847 -0.00735332; … ; 0.0307018 0.0553384 … 0.098546 0.0143964; -0.105442 0.0076203 … -0.0140332 -0.0492025],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("List_List_Pair_Variable_LambdaTerm",200)],Class("List_Pair_Variable_LambdaTerm",100),[0.0298028 -0.00290223 … -0.0244336 0.03121; -0.0246511 0.0841432 … -0.126051 -0.0497068; … ; -0.0296114 -0.14479 … 0.033668 -0.0920792; -0.0348156 0.0464107 … 0.0591687 0.0752593],Transformation(sigmoid,dsigmoid),0.01),DFunction(Class[Class("List_List_Pair_Variable_LambdaTerm",200)],Class("List_List_Pair_Variable_Lambd